In [1]:
import ROOT as r
import math as m
import os

Welcome to JupyROOT 6.18/00


In [2]:
Output = r.TChain("output")
directory = "/global/projecta/projectdirs/atlas/eressegu/Ntuple_h026_0630/data17/data/data17/root"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        Output.Add(f)

i = 0
Output2 = r.TChain("output")
directory2 = "/global/projecta/projectdirs/atlas/eressegu/Ntuple_h026_0630/NonRes/mc16a/NonRes/root"
for filename in os.listdir(directory2):
    i += 1
    if i == 8:
        break
    f = os.path.join(directory2, filename)
    if os.path.isfile(f):
        Output2.Add(f)

In [3]:
photon1 = r.TLorentzVector()
photon2 = r.TLorentzVector()
diphoton = r.TLorentzVector()

h_2017_thetaphi = r.TH2F("2017thetavsphi", "2017thetavsphi", 60,-180, 180, 60, -90, 90)
h_MC_thetaphi = r.TH2F("MCthetavsphi", "MCthetavsphi", 60,-180, 180, 60, -90, 90)
h_MC_M = r.TH1F("h_MC_M", "h_MC_M", 60, 0, 600)
h_2017_M = r.TH1F("h_2017_M", "h_2017_M", 60, 0, 600)

In [4]:
for entry in Output:
    if entry.isPassed:
        photon1.SetPtEtaPhiE(entry.ph_pt1, entry.ph_eta1, entry.ph_phi1, entry.ph_E1)
        photon2.SetPtEtaPhiE(entry.ph_pt2, entry.ph_eta2, entry.ph_phi2, entry.ph_E2)
        diphoton = photon1 + photon2
        angle = diphoton.Pt() / diphoton.Pz()
        actual_angle = m.atan(angle) * 180 / m.pi
        phi = diphoton.Phi() * 180 / m.pi
        h_2017_thetaphi.Fill(phi, actual_angle)
        h_2017_M.Fill(diphoton.M())

Warning in <TClass::Init>: no dictionary for class xAOD::EventFormat_v1 is available
Warning in <TClass::Init>: no dictionary for class xAOD::AuxInfoBase is available
Warning in <TClass::Init>: no dictionary for class SG::IAuxStore is available
Warning in <TClass::Init>: no dictionary for class SG::IConstAuxStore is available
Warning in <TClass::Init>: no dictionary for class SG::IAuxStoreIO is available
Warning in <TClass::Init>: no dictionary for class SG::IAuxStoreHolder is available
Warning in <TClass::Init>: no dictionary for class SG::AuxElement is available
Warning in <TClass::Init>: no dictionary for class SG::IAuxElement is available
Warning in <TClass::Init>: no dictionary for class xAOD::FileMetaData_v1 is available
Warning in <TClass::Init>: no dictionary for class xAOD::FileMetaDataAuxInfo_v1 is available


In [5]:
for entry in Output2:
    if entry.isPassed:
        photon1.SetPtEtaPhiE(entry.ph_pt1, entry.ph_eta1, entry.ph_phi1, entry.ph_E1)
        photon2.SetPtEtaPhiE(entry.ph_pt2, entry.ph_eta2, entry.ph_phi2, entry.ph_E2)
        diphoton = photon1 + photon2
        angle = diphoton.Pt() / diphoton.Pz()
        actual_angle = m.atan(angle) * 180 / m.pi
        phi = diphoton.Phi() * 180 / m.pi
        h_MC_thetaphi.Fill(phi, actual_angle)
        h_MC_M.Fill(diphoton.M())

Warning in <TClass::Init>: no dictionary for class xAOD::TruthMetaData_v1 is available
Warning in <TClass::Init>: no dictionary for class xAOD::TruthMetaDataAuxContainer_v1 is available
Warning in <TClass::Init>: no dictionary for class xAOD::AuxContainerBase is available
Warning in <TClass::Init>: no dictionary for class DataVector<xAOD::TruthMetaData_v1> is available


In [6]:
fit = r.TF1("expofit", "expo", 100, 150)
h_2017_M.Fit(fit, "S", "", 100, 120)

fit2 = r.TF1("expofit", "expo", 100, 150)
h_MC_M.Fit(fit2, "S", "", 100, 120)

 FCN=8.46489e-26 FROM MIGRAD    STATUS=CONVERGED      28 CALLS          29 TOTAL
                     EDM=7.32894e-20    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     6.45637e+00   8.11120e-02  -0.00000e+00   1.57015e-08
   2  Slope        4.41658e-02   7.29465e-04   0.00000e+00   1.00474e-06
 FCN=5.55944e-26 FROM MIGRAD    STATUS=CONVERGED      28 CALLS          29 TOTAL
                     EDM=4.62936e-19    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     7.23863e+00   5.51659e-02  -0.00000e+00   4.73228e-08
   2  Slope        4.49163e-02   4.96045e-04   0.00000e+00   2.52156e-06


Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


In [7]:
N1_2017 = fit.Integral(100, 120)
print(N1_2017)
N3_2017 = fit.Integral(130, 150)
print(N3_2017)
N2_2017 = fit.Integral(120, 130)
print(N2_2017)
N1_MC = fit2.Integral(100, 120)
print(N1_MC)
N3_MC = fit2.Integral(130, 150)
print(N3_MC)
N2_MC = fit2.Integral(120, 130)
print(N2_MC)

1694070.7442710467
6373239.786072997
1603657.395452587
4027084.0567152947
15495207.086752776
3852145.479878418


In [8]:
scale_MC = (N1_2017 + N3_2017) / (N1_MC + N3_MC)
print(scale_MC)

0.4132358477321076


In [9]:
h_2017_M_clone = h_2017_M.Clone("mass_2017_clone")
h_MC_M_clone = h_MC_M.Clone("mass_MC_clone")
h_MC_M_clone.Scale(scale_MC)
canvas_1 = r.TCanvas("higgs_mass_both", "higgs_mass_both")
h_2017_M_clone.Draw()
h_MC_M_clone.Draw("SAME")
h_2017_M_clone.GetXaxis().SetTitle("m_{#gamma#gamma} [GeV]")
h_2017_M_clone.GetYaxis().SetTitle("Entries")
canvas_1.SaveAs("MC2017Masses.png")

canvas_2 = r.TCanvas("mass_subtracted", "mass_subtracted")
h_2017_M_clone.Add(h_MC_M_clone, -1)
h_2017_M_clone.Draw()
h_2017_M_clone.SetLineWidth(2)
h_2017_M_clone.SetLineColor(2)
h_2017_M_clone.SetLineStyle(1)
h_2017_M_clone.SetFillColor(2)
canvas_2.SaveAs("MC2017subtracted.png")

Info in <TCanvas::Print>: png file MC2017Masses.png has been created
Info in <TCanvas::Print>: png file MC2017subtracted.png has been created


In [10]:
canvas_thetaphi = r.TCanvas("higgs_thetaphi", "higgs_thetaphi")
h_2017_thetaphi.Draw("colz")
h_2017_thetaphi.GetXaxis().SetTitle("#phi_{#gamma#gamma}")
h_2017_thetaphi.GetYaxis().SetTitle("#theta_{#gamma#gamma}")
canvas_thetaphi.SaveAs("2017thetaphi.png")

h_2017_thetaphi_clone = h_2017_thetaphi.Clone()
h_MC_thetaphi_clone = h_MC_thetaphi.Clone()
h_MC_thetaphi_clone.Scale(scale_MC)
h_2017_thetaphi_clone.Add(h_MC_thetaphi_clone, -1)
h_2017_thetaphi_clone.Draw("colz")
h_2017_thetaphi_clone.GetXaxis().SetTitle("#phi_{#gamma#gamma}")
h_2017_thetaphi_clone.GetYaxis().SetTitle("#theta_{#gamma#gamma}")
canvas_thetaphi.SaveAs("MC2017thetaphisubtracted.png")

Info in <TCanvas::Print>: png file 2017thetaphi.png has been created
Info in <TCanvas::Print>: png file MC2017thetaphisubtracted.png has been created


In [11]:
canvas_3 = r.TCanvas("mass", "mass")
h_2017_M.SetLineWidth(2)
h_2017_M.SetLineColor(2)
h_2017_M.SetLineStyle(1)
h_2017_M.SetFillColor(2)
h_2017_M.Draw()
h_2017_M.GetXaxis().SetTitle("m_{#gamma#gamma} [GeV]")
h_2017_M.GetYaxis().SetTitle("Entries")
canvas_3.SaveAs("2017_M.png")

Info in <TCanvas::Print>: png file 2017_M.png has been created
